source:

https://www.33rdsquare.com/text-analysis-topic-modeling-with-bert/?utm_source=chatgpt.com

https://link.springer.com/article/10.1007/s41870-023-01268-w?utm_source=chatgpt.com

Datasource: https://www.kaggle.com/datasets/elvinrustam/books-dataset


In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

import nltk
from collections import Counter
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
import torch

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
df = pd.read_csv('../Data/BooksDataset.csv')
df.sample(5)

,Title,Authors,Description,Category,Publisher,Publish Date,Price
17456,"Incidents of Travel in Central America, Chiapa...","By Stephens, John Lloyd","""The ground was entirely new; there were no gu...","Travel , Special Interest , Adventure",Dover Publications,"Sunday, June 1, 1969",Price Starting at $4.99
75237,"6 + 1 Traits of Writing: The Complete Guide, G...","By Culham, Ruth","Ideas, Organization, Voice, Word Choice, Sente...","Education , Teaching Methods & Materials , Ge...",Scholastic Professional Books,"Wednesday, January 1, 2003",Price Starting at $5.49
38132,Neanderthal Book & Skeleton (Hand in Hand with...,"By Cumbaa, Stephen, Lafave, Kim (ILT), Hehner,...",Introducing a caveman for kids. Following the ...,"Juvenile Nonfiction , Social Science , Archae...",Workman Publishing Company,"Wednesday, January 1, 1997",Price Starting at $5.29
24450,Listening to America: Twenty-Five Years in the...,"By Wertheimer, Linda (EDT) and Npr (National P...",The publication of Listening to America coinci...,NaN,Houghton Mifflin,"Monday, May 1, 1995",Price Starting at $6.36
28965,Baby Sister for Herry (Sesame Street Growing Up),"By Kingsley, Emily P.",A little blue monster named Herry finds life i...,NaN,Golden Press,"Wednesday, August 1, 1984",Price Starting at $5.29


In [14]:
df.dropna(subset=['Description'], inplace=True)
df.dropna(subset=['Category'], inplace=True)

df.shape

(65305, 7)

In [3]:
# Keep only the first 500 rows in df
df = df.iloc[:500]

# Rename 'title' column to 'Comment'
df.rename(columns={'Description': 'Comment'}, inplace=True)
df['Comment']

7      Collects poems written by the eleven-year-old ...
8      The Russian author offers an affectionate chro...
10     A humor classic, this tongue-in-cheek diet pla...
11     Deadly germs sprayed in shopping malls, bomb-l...
13     "The Bible and the social and moral consequenc...
                             ...                        
837    In recent years, much attention has been lavis...
838    One hot summer morning, William Hallberg, auth...
840    Nine of America's most influential artists, sc...
842    Love. Sex. Respect. Race. Politics. Unity. Spi...
843    This book is a journey down the Italian penins...
Name: Comment, Length: 500, dtype: object

In [5]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode comments into embeddings
def encode_comments(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

# Encode comments
embeddings = encode_comments(df['Comment'].tolist())


In [7]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Set of English stopwords and initialize lemmatizer
stop_words = set(stopwords.words('english'))
# Include all digits and symbols
custom_stop_words = set(string.digits + string.punctuation) 
additional_stop_words = ['the', 'and', 'was', 'were', 'with', 'a']

stop_words.update(custom_stop_words)
stop_words.update(additional_stop_words)

lemmatizer = WordNetLemmatizer()

def encode_comments(texts):
    # Function to remove stopwords
    def remove_stopwords(text):
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
        return " ".join(filtered_text)

    # Function to remove stopwords and lemmatize
    import string

    def preprocess_text(text):
        # Remove punctuation from text
        translator = str.maketrans('', '', string.punctuation)
        text = text.translate(translator).lower()  # Lowercasing the text here

        word_tokens = word_tokenize(text)
        filtered_and_lemmatized_text = [
            lemmatizer.lemmatize(word)
            for word in word_tokens
            if word not in stop_words
        ]
        return " ".join(filtered_and_lemmatized_text)

    # Remove stopwords from each comment
    cleaned_texts = [preprocess_text(text) for text in texts]

    # Tokenization and encoding
    encoded_input = tokenizer(cleaned_texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

# Encode comments
embeddings = encode_comments(df['Comment'].tolist())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Perform clustering on embeddings (example: K-Means)
num_topics = 5  # Set the number of topics
kmeans = KMeans(n_clusters=num_topics, random_state=0).fit(embeddings.numpy())
df['topic'] = kmeans.labels_

# For Coherence Score, transform BERT embeddings to bag-of-words
texts = [comment.split() for comment in df['Comment'].tolist()]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Use KMeans centroids to find closest comments for each topic
def get_representative_comments(topic_idx, n_representative=10):
    indices = np.where(df['topic'] == topic_idx)[0]
    centroid = kmeans.cluster_centers_[topic_idx]
    distances = cosine_similarity([centroid], embeddings[indices].numpy())
    closest_indices = np.argsort(distances[0])[:n_representative]
    return [df.iloc[indices[i]]['Comment'] for i in closest_indices]

/Users/sajjadislam/opt/anaconda3/envs/py312_topic_modeling/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [9]:
def get_most_frequent_words(comments, top_n=5):
    # Tokenize words, convert to lower case, and filter out stop words and punctuation
    words = [word for comment in comments
             for word in word_tokenize(comment.lower())
             if word not in stop_words and word not in string.punctuation]

    # Find the most common words
    most_common = [word for word, freq in Counter(words).most_common(top_n)]
    return most_common

# Create lists of most frequent words representing each topic
topic_words = [get_most_frequent_words(get_representative_comments(i)) for i in range(num_topics)]
topic_words

[["'s", 'poker', 'world'],
 ['recipes', 'breads', 'de'],
 ['security', 'first', 'published'],
 ["'s", '--', 'company'],
 ['looks', 'describes', 'history']]

In [31]:
# Remove the first row (index 0)
#topic_words = topic_words[1:]

# Output the modified list
#print(topic_words)

In [10]:
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_v')
#cm = CoherenceModel(topics=topic_words, dictionary=dictionary, coherence='u_mass')
coherence_score = cm.get_coherence()
print(f'Coherence Score: {coherence_score}')


Coherence Score: 0.4873080266755908


3 topic =  0.2098919420259906

5 topic = 0.5330362309251153

10 topic = 0.3184375779964808

15 topic = 0.45240816473933504

20 topic = 0.4231468971928173

25 topic = 0.4100195812207564

30 topic = 
